# Knapsack With Integer Weights

Now we have weight $(w_\alpha \geq 0)$ and value $(c_\alpha \geq 0)$ of N items.
We put some of these into a knapsack we keep the limit of weight as $ \displaystyle W(= \sum _ {\alpha = 1} ^ {N} w_{\alpha}x_{\alpha})$ , $W_{limit}$ and take the maximum sum of value $ \displaystyle C(= \sum _ {\alpha = 1} ^ {N} c_{\alpha}x_{\alpha})$.

# Cost function

We have cost function from this paper.
( https://arxiv.org/abs/1302.5843 )。  

$H = H_A + H_B$  
$\displaystyle H_A = A \left( 1 - \sum _ { i = 1 } ^ { W_{limit} } y_i \right) ^ { 2 }+ A \left( \sum _ { i = 1 } ^ { W_{limit} } i y_i - \sum _ { \alpha } w_\alpha x_\alpha \right) ^ { 2 }$  
$\displaystyle H_B = -B \sum _ { \alpha } c_\alpha x_\alpha$  

# Creating QUBO
Now we have the QUBO

$H _ { A }$  
$ \displaystyle
=A \left\{
    -2 \left( \sum _ { i = 1 } ^ { W_{limit} } y_i \right)
    +\left( \sum _ { i = 1 } ^ { W_{limit} } y_i \right) ^ { 2 }
    +\left( \sum _ { i = 1 } ^ { W_{limit} } iy_i \right) ^ { 2 }
    -2 \left( \sum _ { i = 1 } ^ { W_{limit} } iy_i \right) \left( \sum _ \alpha w_\alpha x_\alpha \right)
    +\left( \sum _ \alpha w_\alpha x_\alpha \right) ^ 2 \right\}
$  

$ \displaystyle
= A \left\{
    \left( \sum _ { i = 1 } ^ { W_{limit} } -2 y_i \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } y_i ^ 2 \right)
   +\left( \mathop { \sum \sum } _ { i \neq j } ^ { W_{limit} } 2 y_i y_j \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } i ^ 2 y_i ^ 2 \right)
   \right.
$  

$ \displaystyle
\quad \left.
   +\left( \mathop { \sum \sum } _ {i \neq j } ^ { W_{limit} } 2ij y_i y_j \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } \sum _ { \alpha } \left( -2i w _ \alpha x _ \alpha y _ i \right) \right)
   +\left( \sum _ \alpha w_\alpha ^ 2 x_\alpha ^ 2 \right)
   +\left( \sum _ { \alpha } \sum _ { \beta } 2 w_\alpha w_\beta x_\alpha x_\beta\right )
   \right\}
$  

$ \displaystyle
= A \left\{
     \sum _ { \alpha } w_\alpha ^ 2x_\alpha ^ 2
   +\sum _ { \alpha } \sum _ { \beta } 2 w_\alpha w_\beta x_\alpha x_\beta
   +\sum _ { \alpha } \sum _ { i = 1 } ^ { W_{limit} }\left( -2 w_\alpha i \right) x_\alpha y_i
   +\sum _ { i = 1 } ^ { W_{limit} } \left( i ^ 2 - 1\right) y_i ^ 2
   +\mathop { \sum \sum } _ { i \neq j } ^ { W_{limit} } 2 \left( 1 + ij \right) y_i y_j
   \right\}
$ 

Let's start solving it using blueqat

pip install blueqat

And prepare for the libraries.

In [0]:
import numpy as np
import blueqat.opt as wq

Now we prepare some functions.


In [0]:
class Item():
    def __init__(self, number, weight, cost):
        self.__number = number
        self.__weight = weight
        self.__cost = cost

    @property
    def weight(self):
        return self.__weight

    @property
    def cost(self):
        return self.__cost

    def __str__(self):
        return f"#{self.__number} (weight : {self.weight}, cost : {self.cost})"

In [0]:
def get_qubo(items, wlimit, A, B):
    # qubo
    x_size = len(items)
    y_size = wlimit
    size = x_size + y_size
    qubo = np.zeros((size, size))
    for i in range(0,size):
        for j in range(0, size):
            if i > j:
                continue
                
            wi = items[i].weight if i < x_size else 0
            wj = items[j].weight if j < x_size else 0 
            ci = items[i].cost if i < x_size else 0      
            wsum_i = i - x_size + 1
            wsum_j = j - x_size + 1

            # diagonal elements
            if i == j:
                if i < x_size: # xi*xi
                    qubo[i][i] = A * wi ** 2 - B * ci
                else: # yi*yi
                    qubo[i][i] = A * (-1 + wsum_i * wsum_j)
            # off-diagonal
            else: # i < j
                if i < x_size and j < x_size: # xi*xj
                    qubo[i][j] = 2 * A * wi * wj
                elif i < x_size and j >= x_size: # xi*yj
                    qubo[i][j] = -2 * A * wi * wsum_j
                else: # yi*yj
                    qubo[i][j] = 2 * A * (1 + wsum_i * wsum_j)

    return qubo

In [0]:
def show_answer(q, items):
    print(q)
    answers = []
    weight = 0 
    cost = 0
    for i in range(len(items)):
        if q[i] > 0:
            answers.append(items[i])
            weight += items[i].weight
            cost += items[i].cost
    for answer in answers:
        print(f"selected : {answer}")
    print(f"total weight : {weight}")
    print(f"total cost : {cost}")
    print("")

In [0]:
def annealing(annealer):
    answer_opt = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
    count = 0
    found = False
    for _ in range(100):
        q = annealer.sa()
        count += 1
        print(f"count : {count}")
        show_answer(q, items)
        if q == answer_opt:
            print(f"found in {count} times")
            found = True
            break
    if(found == False):
        print("optimum solution has not been found")


Now we have some example of items.

In [0]:
items = []
items.append(Item(number=0, weight=1, cost=10))
items.append(Item(number=1, weight=2, cost=15))
items.append(Item(number=2, weight=2, cost=55))
items.append(Item(number=3, weight=3, cost=50))
items.append(Item(number=4, weight=4, cost=40))
items.append(Item(number=5, weight=5, cost=50))
items.append(Item(number=6, weight=7, cost=30))
items.append(Item(number=7, weight=8, cost=35))
items.append(Item(number=8, weight=7, cost=60))
items.append(Item(number=9, weight=8, cost=80))

And solve with some parameters.

In [17]:
# max weight
wlimit = 10

# get B from the cost
A = 1
cmax = max(items, key = lambda item : item.cost).cost
B = A / cmax * 0.9

# SA
annealer = wq.opt()
annealer.qubo = get_qubo(items, wlimit, A, B)
annealing(annealer)

count : 1
[0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
selected : #2 (weight : 2, cost : 55)
selected : #3 (weight : 3, cost : 50)
selected : #6 (weight : 7, cost : 30)
selected : #7 (weight : 8, cost : 35)
selected : #8 (weight : 7, cost : 60)
total weight : 27
total cost : 230

count : 2
[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
selected : #0 (weight : 1, cost : 10)
selected : #1 (weight : 2, cost : 15)
selected : #3 (weight : 3, cost : 50)
selected : #5 (weight : 5, cost : 50)
selected : #7 (weight : 8, cost : 35)
total weight : 19
total cost : 160

count : 3
[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
selected : #1 (weight : 2, cost : 15)
selected : #4 (weight : 4, cost : 40)
selected : #7 (weight : 8, cost : 35)
total weight : 14
total cost : 90

count : 4
[1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
selected : #0 (weight : 1, cost : 10)
selected : #3 (weight : 3, cost : 50)
selected : #4 (weight : 4, cost : 40)
s